# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [1]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [2]:
def differences(graph, partition):
    # Разница, которую мы получаем при перенесении вершины в другую часть.
    diff_exch = [0] * len(graph[0])
    for u, v in graph[1]:
        if (u in partition) != (v in partition):
            diff_exch[u - 1] -= 1
            diff_exch[v - 1] -= 1
        else:
            diff_exch[u - 1] += 1
            diff_exch[v - 1] += 1    
    return diff_exch

In [3]:
def best_candidates(graph, partition, fixed_first, fixed_second):
    diff_exch = differences(graph, partition)
    complement = graph[0] - partition
    best_u, best_v, best_diff = -1, -1, 0
    for u in partition - fixed_first:
        for v in complement - fixed_second:
            flag = 0
            # Если было ребро между вершинами из разных частей,
            # то мы ошибочно посчитали его, и даже дважды.
            if (u, v) in graph[1] or (v, u) in graph[1]:
                flag = 1
            new_diff = diff_exch[u - 1] + diff_exch[v - 1] + 2 * flag
            if best_u == -1 or new_diff < best_diff:
                best_u = u
                best_v = v
                best_diff = new_diff
    return best_u, best_v

In [4]:
def variable_depth_local_search(graph, depth):
    starting_point = set(range(1, len(graph[0]) // 2 + 1))
    fixed_first = set([])
    fixed_second = set([])
    for i in range(depth):
        best_u, best_v = best_candidates(graph, starting_point,
                                       fixed_first, fixed_second)
        starting_point.remove(best_u)
        starting_point.add(best_v)
        fixed_first.add(best_v)
        fixed_second.add(best_u)
    return starting_point

In [10]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    depths = [2, 2, 2, 2]
    for filename, depth in zip(filenames, depths):
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance, depth))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [11]:
run_all()

Solving instance add20.graph… done in 1.4 seconds with quality 11098
Solving instance cti.graph… done in 1e+02 seconds with quality 94060
Solving instance t60k.graph…

KeyboardInterrupt: 

## Выводы
(Здесь опишите свои наблюдения и подобранные параметры для каждогр из четырёх входных графов.)